In [1]:
from src.train.ptuning_train import train
import src.benchmarks as benchmarks
import src.prompt.mbpp as mbpp
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel
import src.config as config
import torch

from tqdm.auto import tqdm
import src.test_execute as test_execute
import src.metrics as metrics_utils
import re
import time

Skipping import of cpp extensions due to incompatible torch version 2.9.0+cu128 for torchao version 0.14.0         Please see GitHub issue #2919 for more info


In [2]:
# train()

In [3]:
print("⏳ Загружаем базу...")
base_model = AutoModelForCausalLM.from_pretrained(
    config.MODEL_PATH,
    torch_dtype=torch.bfloat16,
    device_map="auto",
	attn_implementation="sdpa",
    trust_remote_code=True,
)
tokenizer = AutoTokenizer.from_pretrained(config.MODEL_PATH, trust_remote_code=True)

print("🔗 Подключаем P-Tuning адаптер...")
# Путь должен совпадать с тем, куда сохранил train_ptuning.py
ptuning_path = config.PTUNING_MODEL_PATH
model = PeftModel.from_pretrained(base_model, ptuning_path)

print("✅ Готово к тестам!")

`torch_dtype` is deprecated! Use `dtype` instead!


⏳ Загружаем базу...
🔗 Подключаем P-Tuning адаптер...
✅ Готово к тестам!


In [4]:
model.eval()
model = torch.compile(model)

In [5]:
prompt = (
    "<|im_start|>user\n"
    "Write a python function to reverse a string.<|im_end|>\n"
    "<|im_start|>assistant\n"
)

inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

with torch.inference_mode():
    outputs = model.generate(
        **inputs,
        max_new_tokens=1024, # Даем место на подумать

        # === ЛЕКАРСТВО ОТ ЗАЦИКЛИВАНИЯ ===
        do_sample=True,        # Включаем сэмплирование (помогает выйти из цикла)
        temperature=0.1,       # Но держим очень низким, чтобы код был точным
        top_p=0.9,             # Немного свободы
        repetition_penalty=1, # <--- ГЛАВНОЕ: Штраф 20% за повторы
        # ================================

        pad_token_id=tokenizer.eos_token_id,
        eos_token_id=tokenizer.eos_token_id
    )

print("-" * 30)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))
print("-" * 30)

/opt/conda/lib/python3.11/site-packages/peft/peft_model.py:2141: UserWarning: Position ids are not supported for parameter efficient tuning. Ignoring position ids.
  warnings.warn("Position ids are not supported for parameter efficient tuning. Ignoring position ids.")


------------------------------
user
Write a python function to reverse a string.
assistant
<think>
Okay, I need to write a Python function to reverse a string. Let me think. The user wants me to create an expert Python assistant. The task is to write a function to reverse a string. So, the function should take a string as input and return the reversed string. 

First, I should check if the input is empty. If it's empty, return an empty string. Then, reverse the string. But wait, maybe the user wants to reverse the string in a specific way. Oh, the user said "reverse a string", so maybe just reverse the characters. 

Wait, the user might be asking for a function that can reverse any string. So, the function should take a string and return the reversed version. Let me think about the steps. 

In Python, the string method is ''.join(reversed(s)) or s[::-1]. So, the function could be something like def reverse_string(s): return ''.join(reversed(s)). 

But maybe the user wants to handle edg

In [6]:
# 1. Стандартные импорты, необходимые для задач MBPP/HumanEval
# Если модель их не напишет, мы добавим их принудительно.
STANDARD_IMPORTS = """
import math
import cmath
import re
import sys
import heapq
import collections
import itertools
from typing import List, Dict, Tuple, Optional
"""

def extract_code_clean(text):
    """
    Вырезает код из ответа модели.
    Если <think> не закрыт (обрыв генерации), удаляет всё до конца строки.
    """
    # 1. Агрессивно удаляем <think>
    # (?:</think>|$) означает "или закрывающий тег, или конец текста"
    text = re.sub(r"<think>[\s\S]*?(?:</think>|$)", "", text, flags=re.DOTALL)

    # 2. Ищем Markdown блоки кода (```...```)
    code_blocks = re.findall(r"```(?:python)?\n(.*?)```", text, re.DOTALL)

    if code_blocks:
        # Берем самый длинный блок
        clean_code = max(code_blocks, key=len).strip()
    elif "def " in text:
        # Фолбэк: ищем def, но теперь (после шага 1) мы уверены, что это не мысли
        # Но чтобы не захватить мусор перед def, отрезаем всё до него
        start_idx = text.find("def ")
        clean_code = text[start_idx:].strip()
    else:
        clean_code = ""

    # 3. Если код пустой (например, модель не успела его написать из-за лимита токенов)
    # возвращаем пустую строку, чтобы не ломать тесты импортами
    if not clean_code:
        return ""

    # 4. Приклеиваем импорты
    return STANDARD_IMPORTS + "\n" + clean_code


def calculate_entropy_transformers_single(scores, sequences, input_len, batch_idx):
    """
    Считает энтропию для ОДНОГО элемента из батча.
    """
    # Защита от выхода за границы, если генерация сломалась
    if batch_idx >= sequences.shape[0]: return 0.0

    generated_ids = sequences[batch_idx, input_len:]
    log_probs_sum = 0.0
    count = 0

    for i, step_logits in enumerate(scores):
        if i >= len(generated_ids): break

        # step_logits: [Batch, Vocab]
        sample_logits = step_logits[batch_idx]
        step_logprobs = torch.nn.functional.log_softmax(sample_logits, dim=-1)

        chosen_token_id = generated_ids[i]
        token_logprob = step_logprobs[chosen_token_id].item()

        log_probs_sum += token_logprob
        count += 1

    if count == 0: return 0.0
    return - (log_probs_sum / count)


def run_ptuning_mbpp(dataset_split, model, tokenizer, batch_size=8, limit=None):
    print(f"🚀 Запуск P-Tuning бенчмарка...")

    tokenizer.padding_side = "left"
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token

    metrics_all = []
    outputs_text = []

    dataset = dataset_split
    if limit:
        dataset = dataset.select(range(limit))

    try:
        # <--- 2. Создаем объект pbar для управления полоской
        pbar = tqdm(range(0, len(dataset), batch_size), desc="Benchmarking")

        for i in pbar:
            try:
                # 1. Подготовка батча
                batch_slice = dataset.select(range(i, min(i + batch_size, len(dataset))))
                prompts_text = [
                    mbpp.build_prompt(ex, tokenizer, train=False)["text"]
                    for ex in batch_slice
                ]

                inputs = tokenizer(
                    prompts_text,
                    return_tensors="pt",
                    padding=True,
                    truncation=True,
                    max_length=2048
                ).to("cuda")

                input_len = inputs.input_ids.shape[1]

                # 2. ГЕНЕРАЦИЯ с замером времени
                t_start = time.time()  # <--- 3. Старт таймера

                with torch.no_grad():
                    outputs = model.generate(
                        **inputs,
                        max_new_tokens=2048,
                        do_sample=False,
                        repetition_penalty=1.0,
                        pad_token_id=tokenizer.eos_token_id,
                        eos_token_id=tokenizer.eos_token_id,
                        return_dict_in_generate=True,
                        output_scores=True
                    )

                t_end = time.time()  # <--- 4. Стоп таймера

                # <--- 5. Расчет скорости
                # Количество новых токенов в одном сэмпле (с учетом паддинга до макс. длины в батче)
                new_tokens_per_sample = outputs.sequences.shape[1] - input_len
                # Общее число токенов в батче (GPU обрабатывала их все параллельно)
                total_tokens_gen = new_tokens_per_sample * outputs.sequences.shape[0]

                duration = t_end - t_start
                if duration > 0:
                    speed = total_tokens_gen / duration
                    # Обновляем инфо в баре
                    pbar.set_postfix({"speed": f"{speed:.2f} tok/s", "len": new_tokens_per_sample})

                batch_sequences = outputs.sequences
                batch_scores = outputs.scores

                # 3. Обработка результатов
                for j, seq in enumerate(batch_sequences):
                    generated_tokens = seq[input_len:]
                    raw_answer = tokenizer.decode(generated_tokens, skip_special_tokens=True)

                    code_to_test = extract_code_clean(raw_answer)
                    outputs_text.append(code_to_test)

                    if i == 0 and j == 0:
                        print("-" * 20)
                        print(f"DEBUG EXTRACTED:\n{code_to_test[:200]}...")
                        print("-" * 20)

                    example = batch_slice[j]
                    tests = example["test_list"]
                    passed, total = test_execute.run_mbpp_tests_for_sample(code_to_test, tests)

                    entropy = calculate_entropy_transformers_single(batch_scores, batch_sequences, input_len, j)

                    metrics_all.append({
                        "pass@1": 1 if passed == total and total > 0 else 0,
                        "%passed": passed / total if total > 0 else 0,
                        "entropy": entropy
                    })

                # 4. ОЧИСТКА ПАМЯТИ
                del inputs, outputs, batch_sequences, batch_scores
                torch.cuda.empty_cache()

            except Exception as e:
                print(f"\n⚠️ Ошибка в батче {i}: {e}")
                continue

    except KeyboardInterrupt:
        print("\n🛑 Остановили вручную. Сохраняем, что есть...")

    return metrics_all, outputs_text

In [7]:
# Загружаем датасет (validation или test)
# Используем твой же mbpp.py
mbpp_data = mbpp.get_dataset()["test"] # или "test"

# Запускаем (ограничим 20 примерами для быстрой проверки, убери limit=20 для полного прогона)
metrics_result, codes = run_ptuning_mbpp(mbpp_data, model, tokenizer, batch_size=4, limit=None)

# Агрегируем результаты твоей функцией
final_stats = metrics_utils.aggregate_metrics(metrics_result)

print("\n" + "="*30)
print(f"📊 ИТОГОВЫЕ МЕТРИКИ P-TUNING:")
print(f"Pass@1:    {final_stats['mean_pass@1']:.2%}")
print(f"Avg Passed:{final_stats['mean_%_passed']:.2%}")
print(f"Samples:   {final_stats['total_samples']}")
print("="*30)

🚀 Запуск P-Tuning бенчмарка...


Benchmarking:   0%|          | 0/65 [00:00<?, ?it/s]

The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


--------------------
DEBUG EXTRACTED:

import math
import cmath
import re
import sys
import heapq
import collections
import itertools
from typing import List, Dict, Tuple, Optional

def remove_Occ(s, ch):
    if ch not in s:
        retur...
--------------------

📊 ИТОГОВЫЕ МЕТРИКИ P-TUNING:
Pass@1:    40.47%
Avg Passed:44.39%
Samples:   257
